In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
api_key = None
secret_key = None
TRADE_API_KEY=api_key
TRADE_API_SECRET=secret_key

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

import os

if api_key is None:
    TRADE_API_KEY = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    TRADE_API_SECRET = os.environ.get('ALPACA_SECRET_KEY')

In [2]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [3]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest    
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus 
)
from alpaca.common.exceptions import APIError
import time
import asyncio
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import nest_asyncio
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.trading.models import TradeUpdate
from alpaca.trading.client import TradingClient
from alpaca.trading.stream import TradingStream
from alpaca.trading.requests import GetOptionContractsRequest, MarketOrderRequest
from alpaca.trading.enums import AssetStatus, ContractType, AssetClass
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()
# check version of alpaca-py
alpaca.__version__

'0.37.0'

In [4]:
# Initialize Alpaca clients

trading_client = TradingClient(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET, paper=paper)
trade_update_stream = TradingStream(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET, paper=paper)
stock_data_client = StockHistoricalDataClient(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET)
option_data_client = OptionHistoricalDataClient(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET)

In [5]:
# Configuration

underlying_symbol = "NVDA"
max_abs_notional_delta = 200
risk_free_rate = 0.045
positions = {}
stock_trades = {
    'avg_price': 0.0,
    'total_shares': 0.0,
    'realized_pnl': 0.0
}

In [9]:
#liquidate existing positions
def get_unrealized_pl():
  print(f"get_unrealized_pl {underlying_symbol}")
  equity_pl =0.0
  option_pl = 0.0 
  all_positions = trading_client.get_all_positions()
  current_entry_price = 0.0
  # print(all_positions)
  for p in all_positions:
        if(underlying_symbol not in p.symbol):
            continue
        if p.asset_class == AssetClass.US_OPTION:   
          option_pl += float(p.unrealized_pl)
          # print(p)
        elif p.asset_class == AssetClass.US_EQUITY or p.asset_class == AssetClass.CRYPTO:
          equity_pl =float(p.unrealized_pl)
          current_entry_price=float(p.avg_entry_price)
        # print(p)
  print(f"current_entry_price: ${current_entry_price:.2f}")
  print(f"Equity P&L: ${equity_pl:.2f}")
  print(f"Option P&L: ${option_pl:.2f}")
  print(f"Total P&L: ${equity_pl + option_pl:.2f}")
get_unrealized_pl()

get_unrealized_pl NVDA
current_entry_price: $122.46
Equity P&L: $-613.29
Option P&L: $589.00
Total P&L: $-24.29


: 